In [ ]:
'''
Train data 2100 ,Test data 900 & Val  data 1000
epochs 10
Used EfficientNet series models
Ensemble Accuracy 53%
'''

In [2]:
#content google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!nvidia-smi

Mon Nov 14 06:34:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import SGD
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install tensorflow-addons
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
import glob
#select tensorflow's version
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 3.1 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [5]:
#形状設定
IMAGE_SHAPE=(224,224)

#リアルタイムにデータ拡張しながら，テンソル画像データのバッチを生成する。(1/255 メモリ消費の削減)
image_generator=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
    )

In [6]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [7]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
batch_size=32

train_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='training',
)

val_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='validation'
)

Found 8471 images belonging to 4 classes.
Found 3630 images belonging to 4 classes.


In [8]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
#train_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/train',target_size=IMAGE_SHAPE)
test_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/test',target_size=IMAGE_SHAPE)
gizi_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/train',target_size=IMAGE_SHAPE)
val_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/val',target_size=IMAGE_SHAPE)

Found 4010 images belonging to 4 classes.
Found 12101 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [9]:
#各データセットの画像バッチ形状と、ラベルのバッチ形状を表示する
#擬似データはラベル付けない
for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_val_batch,label_val_batch in val_data:
  print("Image batch shape:",image_val_batch.shape)
  print("Label batch shape:",label_val_batch.shape)
  break

for image_test_batch,label_test_batch in test_data:
  print("Image batch shape:",image_test_batch.shape)
  print("Label batch shape:",label_test_batch.shape)
  break

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)


In [10]:
#モデル設定
MODEL_DIR='/content/drive/MyDrive/result7'
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)

checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )
#filepath:モデルファイルを保存する場所を選択
#monitor:監視するメトリック名
#mode:ファイルを上書きするかどうか
#save_weights_only:モデルのウェイトのみを保存する
#verbose:1、冗長モード                                             

In [11]:
checkpoint1=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model01-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [12]:
checkpoint2=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model02-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [13]:
checkpoint3=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model03-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [14]:
#訓練プロセスの可視化
#カスタムコールバックを作成して、すべてのエポックでのトレーニングの進行状況を視覚化します
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses=[]
    self.batch_acc=[]
    
  def on_train_batch_end(self,batch,logs=1e-12):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [15]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetv2
feature_extractor_url="https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer=hub.KerasLayer(feature_extractor_url,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer.trainable=False

#上位の分類層を接合する
ebc1=Sequential([feature_extractor_layer,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [16]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb1
feature_extractor_url_b1="https://tfhub.dev/google/efficientnet/b1/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b1=hub.KerasLayer(feature_extractor_url_b1,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b1.trainable=False

#上位の分類層を接合する
ebc2=Sequential([feature_extractor_layer_b1,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              6575232   
                                                                 
 dense_1 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 6,580,356
Trainable params: 5,124
Non-trainable params: 6,575,232
_________________________________________________________________


In [17]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb2
feature_extractor_url_b2="https://tfhub.dev/google/efficientnet/b2/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b2=hub.KerasLayer(feature_extractor_url_b2,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b2.trainable=False

#上位の分類層を接合する
ebc3=Sequential([feature_extractor_layer_b2,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1408)              7768562   
                                                                 
 dense_2 (Dense)             (None, 4)                 5636      
                                                                 
Total params: 7,774,198
Trainable params: 5,636
Non-trainable params: 7,768,562
_________________________________________________________________


In [18]:
train_datagenerator = ImageDataGenerator(rescale = 1.0/255,
                                        shear_range = 0.2,
                                        zoom_range = 0.5,
                                        horizontal_flip = True,
                                        rotation_range=10,
                                        width_shift_range=0.2,
                                        brightness_range=[0.2,1.2]
                                        )
test_datagenerator = ImageDataGenerator(rescale = 1.0/255)

In [19]:
training_set = train_datagenerator.flow_from_directory('/content/drive/MyDrive/data/train/',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

Found 12101 images belonging to 4 classes.


In [20]:
class_dict = training_set.class_indices
print(class_dict)

li = list(class_dict.keys())
print(li)

{'CNV_train': 0, 'DME_train': 1, 'DRUSEN_train': 2, 'NORMAL_train': 3}
['CNV_train', 'DME_train', 'DRUSEN_train', 'NORMAL_train']


In [21]:
# predicting an image
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

class votingClassifer:
    'votingClassifer class'


    def __init__(self, estimators,mode="hard",weight=None,show_info="percent"):
        
        if mode not in ["hard","soft"]:
            raise Exception("the mode should be 'hard' or 'soft'")
            
        self.estimators=estimators
        self.mode=mode
        self.weight=weight
        self.show_info=show_info  
            
    def predict(self,x_test):        
        if self.mode=="hard":
            return self.votingClassifer_hard(self.estimators, x_test, show_info=self.show_info)
        else:
            return self.votingClassifer_soft(self.estimators, x_test, weight=self.weight, show_info=self.show_info)

    def getNumberDiff(self, index_classes, n):  
        
        for x in index_classes:
            if index_class_prefer != x:
                return x    
    
    def getNumberElse(self, index_classes, n):

        indices = [i for i, value in enumerate(index_classes) if value != n]
        counts = np.bincount(indices)
        ind=np.argmax(counts)
        n1=(indices == ind).sum()
 
                
        return n1
    
    # hard voting
    def votingClassifer_hard(self,estimators,x_test,show_info='percent'):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")            
            
        
        cpt=0
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs=[],[],[]
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                probabilty = y_prob.flatten()
                max_prob = probabilty.max()
                y_classes = y_prob.argmax(axis=-1)
                index_class, class_name, prob = y_classes,li[y_classes],max_prob
                index_classes.append(index_class)
                class_names.append(class_name)
                probs.append(prob)
            index_classes, class_names, probs = np.array(index_classes), np.array(class_names), np.array(probs)    
            counts = np.bincount(index_classes)
            index_class_prefer=np.argmax(counts)
 
            n1=(index_classes == index_class_prefer).sum()
    
            if n1 == 1:
                print("\n Each estimator predict a different class")
                prob = probs.max()
                indice = [i for i, value in enumerate(probs) if value == prob][0]
                class_name = class_names[indice]

                
            elif n1 == len(estimators)/2 and len(estimators)/2 == self.getNumberElse(index_classes, index_class_prefer):
                
                print("\n the half-estimators predict a class and the other estimators predict a different class")
                
                indices1 = [i for i, value in enumerate(index_classes) if value == index_class_prefer]
                sum2=0
                for ind in indices1:
                    sum2+=probs[ind]
                
                prob1=sum2/len(indices1)
                
                n2=self.getNumberDiff(index_classes, index_class_prefer)      
                
                indices2 = [i for i, value in enumerate(index_classes) if value == n2]
                sum2=0
                for ind in indices2:
                    sum2+=probs[ind]
                
                prob2=sum2/len(indices2)
                
                if prob1 < prob2:
                    prob=prob2
                    indice = [i for i, value in enumerate(index_classes) if value == n2][0]
                    class_name = class_names[indice]
                    
                else:
                    prob=prob1
                    indice = [i for i, value in enumerate(index_classes) if value == index_class_prefer][0]
                    class_name = class_names[indice]
                
            else:
                
                sum1=0
                nbr=0
                for i, index in zip(range(len(probs)),index_classes):
                    if index_class_prefer== index:
                        sum1+=probs[i]
                        nbr+=1
                        
                prob=sum1/nbr
                indice = index_class_prefer
                class_name = li[index_class_prefer]

            
            if show_info=="info":
                cpt+=1
                print("\rpercent: {:.2f}%, li[{}]:{} --> {}".format(cpt*100/N,index_class_prefer, li[index_class_prefer], prob), end='')   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')
          
                
            index_classes_glob.append(indice)
            class_names_glob.append(class_name)
            probs_glob.append(prob)

        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob) 
    
   # soft voting 
    def votingClassifer_soft(self, estimators, x_test, weight=None, show_info="percent"):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")  

        if weight is None :
            weight=np.ones(len(estimators))

        if len(weight) != len(estimators):
            raise Exception("number of models and wheight should be equals")    

        cpt=0 
        # get number of classes 
        x = image.img_to_array(x_test[0])
        x = np.expand_dims(x, axis=0)
        y_prob = estimators[0].predict(x) 
        num_classes =  y_prob.shape[1]

        Tab=np.zeros(num_classes)# num_classes
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs= [], [], []
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                idxs = np.argsort(y_prob)
                # loop over the indexes of the high confidence class labels
                for (index, value) in enumerate(idxs):
                    # build the label and draw the label on the image
                    #label = "{}) {}[{}]: {:.2f}%".format(index,li[value],value, y_prob[value] * 100)
                    #print(label)
                    Tab[value]=y_prob[value]

                probs.append(Tab)

            probs=np.array(probs) # probs.shape: (3, 38)

            proba=[]   

            div=sum(weight)
            for i in range(len(probs[0])):
                s=0
                for j in range(len(weight)):
                    s+= probs[j][i] * weight[j] 
                s=s/div
                proba.append(s)

            proba=np.array(proba)
            max_prob=max(proba)

            indices=[i for i, value in enumerate(proba) if value == max_prob]

            index_class, class_name, prob=indices[0],li[indices[0]],max_prob 

            index_classes_glob.append(index_class)
            class_names_glob.append(class_name)
            probs_glob.append(prob) 

            if show_info=="info":
                cpt+=1
                print("percent: {:.2f}%, li[{}]:{} --> {}%".format(cpt*100/N,index_class, li[index_class], prob))   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')



        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob)  

In [22]:
valid_set = test_datagenerator.flow_from_directory('/content/drive/MyDrive/data/test',
                                                  target_size = (512,512),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

# valid_set
x_test, y_test = valid_set.next() 

Found 4010 images belonging to 4 classes.


In [23]:
batch_stats_callback=CollectBatchStats()
batch_size=32
epochz=5
epoch=10
#steps_per_epoch
s_p_e=np.ceil(train_generator.samples/batch_size)

model1

In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/5
265/265 [==============================] - 2133s 8s/step - loss: 0.4499 - acc: 0.8438
Epoch 2/5
265/265 [==============================] - 151s 571ms/step - loss: 0.3165 - acc: 0.8438
Epoch 3/5
265/265 [==============================] - 151s 570ms/step - loss: 0.1267 - acc: 0.9688
Epoch 4/5
265/265 [==============================] - 151s 570ms/step - loss: 0.2091 - acc: 0.9375
Epoch 5/5
265/265 [==============================] - 151s 570ms/step - loss: 0.3323 - acc: 0.8750


教師モデル学習

In [24]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.4307 - acc: 0.8125
Epoch 1: saving model to /content/drive/MyDrive/result7/model01-01.h5
265/265 [==============================] - 1135s 4s/step - loss: 0.4307 - acc: 0.8125 - val_loss: 0.5272 - val_acc: 0.8036
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.2953 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model01-02.h5
265/265 [==============================] - 229s 864ms/step - loss: 0.2953 - acc: 0.9062 - val_loss: 0.5420 - val_acc: 0.7917
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.2359 - acc: 0.9062
Epoch 3: saving model to /content/drive/MyDrive/result7/model01-03.h5
265/265 [==============================] - 243s 919ms/step - loss: 0.2359 - acc: 0.9062 - val_loss: 0.4825 - val_acc: 0.8298
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.2197 - acc: 0.9062
Epoch 4: saving model to /content/drive/MyDrive/result7/m

In [25]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#擬似ラベル作成
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.2509 - acc: 0.9375
Epoch 1: saving model to /content/drive/MyDrive/result7/model01-01.h5
265/265 [==============================] - 389s 1s/step - loss: 0.2509 - acc: 0.9375 - val_loss: 0.3982 - val_acc: 0.8593
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.2627 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model01-02.h5
265/265 [==============================] - 386s 1s/step - loss: 0.2627 - acc: 0.9062 - val_loss: 0.3473 - val_acc: 0.8774
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.3562 - acc: 0.9062
Epoch 3: saving model to /content/drive/MyDrive/result7/model01-03.h5
265/265 [==============================] - 382s 1s/step - loss: 0.3562 - acc: 0.9062 - val_loss: 0.3066 - val_acc: 0.8907
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.2274 - acc: 0.9062
Epoch 4: saving model to /content/drive/MyDrive/result7/model01-

In [ ]:
#疑似ラベルの予測
gizi_label1=ebc1.predict(gizi_data)

379/379 [==============================] - 216s 560ms/step


In [26]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model1=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model1=Sequential([stochastic_Depth])

#モデルノイズ（dropout）の設定と適用
model1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                           
                  ])

In [27]:
#model1
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model1.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_5 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#model1
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/5
379/379 [==============================] - ETA: 0s - loss: 0.4958 - acc: 0.8438
Epoch 1: saving model to /content/drive/MyDrive/result7/model01-01.h5
379/379 [==============================] - 306s 788ms/step - loss: 0.4958 - acc: 0.8438 - val_loss: 0.3762 - val_acc: 0.8634
Epoch 2/5
379/379 [==============================] - ETA: 0s - loss: 0.2086 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model01-02.h5
379/379 [==============================] - 295s 779ms/step - loss: 0.2086 - acc: 0.9062 - val_loss: 0.3733 - val_acc: 0.8570
Epoch 3/5
379/379 [==============================] - ETA: 0s - loss: 0.2287 - acc: 0.9688
Epoch 3: saving model to /content/drive/MyDrive/result7/model01-03.h5
379/379 [==============================] - 294s 776ms/step - loss: 0.2287 - acc: 0.9688 - val_loss: 0.2860 - val_acc: 0.9011
Epoch 4/5
379/379 [==============================] - ETA: 0s - loss: 0.0959 - acc: 0.9688
Epoch 4: saving model to /content/drive/MyDrive/result7

1

In [28]:
history=model1.fit(train_generator,
                   epochs=epoch,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

#y_prob = model1.predict(val_data)
#y_pred1 = y_prob.argmax(axis=-1)
#y_test1=np.argmax(y_test, axis=1)
# accuracy
#print(model1+" accuracy: ",accuracy_score(y_test1,y_pred1))
#del model1 , y_pred1, y_test1

Epoch 1/10
265/265 [==============================] - ETA: 0s - loss: 0.3162 - acc: 0.9062
Epoch 1: saving model to /content/drive/MyDrive/result7/model01-01.h5
265/265 [==============================] - 3287s 12s/step - loss: 0.3162 - acc: 0.9062 - val_loss: 0.5616 - val_acc: 0.7926
Epoch 2/10
265/265 [==============================] - ETA: 0s - loss: 0.4011 - acc: 0.8438
Epoch 2: saving model to /content/drive/MyDrive/result7/model01-02.h5
265/265 [==============================] - 219s 828ms/step - loss: 0.4011 - acc: 0.8438 - val_loss: 0.5407 - val_acc: 0.8022
Epoch 3/10
265/265 [==============================] - ETA: 0s - loss: 0.3114 - acc: 0.9062
Epoch 3: saving model to /content/drive/MyDrive/result7/model01-03.h5
265/265 [==============================] - 218s 824ms/step - loss: 0.3114 - acc: 0.9062 - val_loss: 0.5418 - val_acc: 0.7975
Epoch 4/10
265/265 [==============================] - ETA: 0s - loss: 0.1871 - acc: 0.9688
Epoch 4: saving model to /content/drive/MyDrive/resu

OSError: ignored

In [ ]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [29]:
model1.save('/content/drive/MyDrive/result7/model1.h5/')

In [ ]:
#from google.colab import runtime
#runtime.unassign()

model2

In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/5
265/265 [==============================] - 60s 225ms/step - loss: 0.6103 - acc: 0.7812
Epoch 2/5
265/265 [==============================] - 46s 174ms/step - loss: 0.4848 - acc: 0.9062
Epoch 3/5
265/265 [==============================] - 46s 172ms/step - loss: 0.3067 - acc: 0.9062
Epoch 4/5
265/265 [==============================] - 47s 177ms/step - loss: 0.2994 - acc: 0.9062
Epoch 5/5
265/265 [==============================] - 46s 173ms/step - loss: 0.3883 - acc: 0.9062


教師モデル学習

In [ ]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              207615832 
                                                                 
 dense_12 (Dense)            (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.2647 - acc: 0.9375
Epoch 1: saving model to /content/drive/MyDrive/result7/model02-01.h5
265/265 [==============================] - 223s 811ms/step - loss: 0.2647 - acc: 0.9375 - val_loss: 0.5438 - val_acc: 0.7970
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.2187 - acc: 0.9375
Epoch 2: saving model to /content/drive/MyDrive/result7/model02-02.h5
265/265 [==============================] - 233s 881ms/step - loss: 0.2187 - acc: 0.9375 - val_loss: 0.4928 - val_acc: 0.8157
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.2958 - acc: 0.8750
Epoch 3: saving model to /content/drive/MyDrive/result7/model02-03.h5
265/265 [==============================] - 211s 795ms/step - loss: 0.2958 - acc: 0.8750 - val_loss: 0.4664 - val_acc: 0.8259
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.1512 - acc: 0.9688
Epoch 4: saving model to /content/drive/MyDrive/result7

In [ ]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              207615832 
                                                                 
 dense_13 (Dense)            (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#擬似ラベル作成
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.2666 - acc: 0.8750
Epoch 1: saving model to /content/drive/MyDrive/result7/model02-01.h5
265/265 [==============================] - 415s 2s/step - loss: 0.2666 - acc: 0.8750 - val_loss: 0.3851 - val_acc: 0.8703
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.3148 - acc: 0.8438
Epoch 2: saving model to /content/drive/MyDrive/result7/model02-02.h5
265/265 [==============================] - 407s 2s/step - loss: 0.3148 - acc: 0.8438 - val_loss: 0.3367 - val_acc: 0.8815
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.4747 - acc: 0.8125
Epoch 3: saving model to /content/drive/MyDrive/result7/model02-03.h5
265/265 [==============================] - 408s 2s/step - loss: 0.4747 - acc: 0.8125 - val_loss: 0.3423 - val_acc: 0.8751
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.1780 - acc: 0.9062
Epoch 4: saving model to /content/drive/MyDrive/result7/model02-

In [ ]:
gizi_label2=ebc2.predict(gizi_data)

379/379 [==============================] - 206s 531ms/step


In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model2=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model2=Sequential([stochastic_Depth])



model2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])



In [ ]:
#model2
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model2.summary()


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              207615832 
                                                                 
 dense_14 (Dense)            (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#model2
history=model2.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )


Epoch 1/5
379/379 [==============================] - ETA: 0s - loss: 0.3168 - acc: 0.8438
Epoch 1: saving model to /content/drive/MyDrive/result7/model02-01.h5
379/379 [==============================] - 342s 879ms/step - loss: 0.3168 - acc: 0.8438 - val_loss: 0.3828 - val_acc: 0.8647
Epoch 2/5
379/379 [==============================] - ETA: 0s - loss: 0.2607 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model02-02.h5
379/379 [==============================] - 329s 868ms/step - loss: 0.2607 - acc: 0.9062 - val_loss: 0.3680 - val_acc: 0.8565
Epoch 3/5
379/379 [==============================] - ETA: 0s - loss: 0.3330 - acc: 0.9062
Epoch 3: saving model to /content/drive/MyDrive/result7/model02-03.h5
379/379 [==============================] - 328s 867ms/step - loss: 0.3330 - acc: 0.9062 - val_loss: 0.2872 - val_acc: 0.8942
Epoch 4/5
379/379 [==============================] - ETA: 0s - loss: 0.2388 - acc: 0.9688
Epoch 4: saving model to /content/drive/MyDrive/result7

2

In [ ]:
history=model2.fit(train_generator,
                   epochs=epoch,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

Epoch 1/10
265/265 [==============================] - ETA: 0s - loss: 0.0601 - acc: 1.0000
Epoch 1: saving model to /content/drive/MyDrive/result7/model02-01.h5
265/265 [==============================] - 267s 1s/step - loss: 0.0601 - acc: 1.0000 - val_loss: 0.3078 - val_acc: 0.8854
Epoch 2/10
265/265 [==============================] - ETA: 0s - loss: 0.1940 - acc: 0.9375
Epoch 2: saving model to /content/drive/MyDrive/result7/model02-02.h5
265/265 [==============================] - 261s 985ms/step - loss: 0.1940 - acc: 0.9375 - val_loss: 0.3467 - val_acc: 0.8601
Epoch 3/10
265/265 [==============================] - ETA: 0s - loss: 0.0809 - acc: 0.9688
Epoch 3: saving model to /content/drive/MyDrive/result7/model02-03.h5
265/265 [==============================] - 265s 1000ms/step - loss: 0.0809 - acc: 0.9688 - val_loss: 0.3249 - val_acc: 0.8747
Epoch 4/10
265/265 [==============================] - ETA: 0s - loss: 0.1044 - acc: 0.9688
Epoch 4: saving model to /content/drive/MyDrive/resul

In [ ]:
model2.save('/content/drive/MyDrive/result7/model2.h5/')

model3

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/5
265/265 [==============================] - 67s 223ms/step - loss: 0.4905 - acc: 0.9062
Epoch 2/5
265/265 [==============================] - 47s 178ms/step - loss: 0.2873 - acc: 0.9688
Epoch 3/5
265/265 [==============================] - 47s 176ms/step - loss: 0.4026 - acc: 0.8750
Epoch 4/5
265/265 [==============================] - 47s 177ms/step - loss: 0.1782 - acc: 0.9688
Epoch 5/5
265/265 [==============================] - 46s 174ms/step - loss: 0.2010 - acc: 0.9688


In [ ]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


教師モデル作成

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.5557 - acc: 0.7812
Epoch 1: saving model to /content/drive/MyDrive/result7/model03-01.h5
265/265 [==============================] - 225s 817ms/step - loss: 0.5557 - acc: 0.7812 - val_loss: 0.5367 - val_acc: 0.7887
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.2924 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model03-02.h5
265/265 [==============================] - 213s 803ms/step - loss: 0.2924 - acc: 0.9062 - val_loss: 0.5040 - val_acc: 0.8116
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.1219 - acc: 0.9688
Epoch 3: saving model to /content/drive/MyDrive/result7/model03-03.h5
265/265 [==============================] - 212s 797ms/step - loss: 0.1219 - acc: 0.9688 - val_loss: 0.4664 - val_acc: 0.8262
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.4089 - acc: 0.8750
Epoch 4: saving model to /content/drive/MyDrive/result7

In [ ]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#擬似ラベル作成
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

Epoch 1/5
265/265 [==============================] - ETA: 0s - loss: 0.1959 - acc: 0.9688
Epoch 1: saving model to /content/drive/MyDrive/result7/model03-01.h5
265/265 [==============================] - 427s 2s/step - loss: 0.1959 - acc: 0.9688 - val_loss: 0.4137 - val_acc: 0.8555
Epoch 2/5
265/265 [==============================] - ETA: 0s - loss: 0.1222 - acc: 1.0000
Epoch 2: saving model to /content/drive/MyDrive/result7/model03-02.h5
265/265 [==============================] - 411s 2s/step - loss: 0.1222 - acc: 1.0000 - val_loss: 0.3514 - val_acc: 0.8727
Epoch 3/5
265/265 [==============================] - ETA: 0s - loss: 0.2853 - acc: 0.8750
Epoch 3: saving model to /content/drive/MyDrive/result7/model03-03.h5
265/265 [==============================] - 415s 2s/step - loss: 0.2853 - acc: 0.8750 - val_loss: 0.3241 - val_acc: 0.8844
Epoch 4/5
265/265 [==============================] - ETA: 0s - loss: 0.1524 - acc: 0.9375
Epoch 4: saving model to /content/drive/MyDrive/result7/model03-

In [ ]:
gizi_label3=ebc3.predict(gizi_data)

379/379 [==============================] - 206s 532ms/step


In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model3=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model3=Sequential([stochastic_Depth])

model3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])

In [ ]:
#model3
model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_5 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
#model3
history=model3.fit(gizi_data,
                   epochs=epochz,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

Epoch 1/5
379/379 [==============================] - ETA: 0s - loss: 0.6537 - acc: 0.7812
Epoch 1: saving model to /content/drive/MyDrive/result7/model03-01.h5
379/379 [==============================] - 2583s 7s/step - loss: 0.6537 - acc: 0.7812 - val_loss: 0.4663 - val_acc: 0.8077
Epoch 2/5
379/379 [==============================] - ETA: 0s - loss: 0.4203 - acc: 0.7812
Epoch 2: saving model to /content/drive/MyDrive/result7/model03-02.h5
379/379 [==============================] - 318s 840ms/step - loss: 0.4203 - acc: 0.7812 - val_loss: 0.3320 - val_acc: 0.8755
Epoch 3/5
379/379 [==============================] - ETA: 0s - loss: 0.1206 - acc: 1.0000
Epoch 3: saving model to /content/drive/MyDrive/result7/model03-03.h5
379/379 [==============================] - 314s 828ms/step - loss: 0.1206 - acc: 1.0000 - val_loss: 0.2803 - val_acc: 0.9022
Epoch 4/5
379/379 [==============================] - ETA: 0s - loss: 0.4027 - acc: 0.8438
Epoch 4: saving model to /content/drive/MyDrive/result7/m

3

In [ ]:
history=model3.fit(train_generator,
                   epochs=epoch,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

Epoch 1/10
265/265 [==============================] - ETA: 0s - loss: 0.1889 - acc: 0.9375
Epoch 1: saving model to /content/drive/MyDrive/result7/model03-01.h5
265/265 [==============================] - 253s 957ms/step - loss: 0.1889 - acc: 0.9375 - val_loss: 0.2811 - val_acc: 0.8928
Epoch 2/10
265/265 [==============================] - ETA: 0s - loss: 0.2104 - acc: 0.9062
Epoch 2: saving model to /content/drive/MyDrive/result7/model03-02.h5
265/265 [==============================] - 249s 941ms/step - loss: 0.2104 - acc: 0.9062 - val_loss: 0.2989 - val_acc: 0.8887
Epoch 3/10
265/265 [==============================] - ETA: 0s - loss: 0.2611 - acc: 0.9375
Epoch 3: saving model to /content/drive/MyDrive/result7/model03-03.h5
265/265 [==============================] - 250s 944ms/step - loss: 0.2611 - acc: 0.9375 - val_loss: 0.4073 - val_acc: 0.8424
Epoch 4/10
265/265 [==============================] - ETA: 0s - loss: 0.1518 - acc: 0.9375
Epoch 4: saving model to /content/drive/MyDrive/res

In [ ]:
#モデル保存
model3.save('/content/drive/MyDrive/result7/model3.h5/')

Accuracy for each model

In [ ]:
from tensorflow import keras

In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/result7/model1.h5/')
model2 = keras.models.load_model('/content/drive/MyDrive/result7/model2.h5/')
model3 = keras.models.load_model('/content/drive/MyDrive/result7/model3.h5/')

In [ ]:
from sklearn.base import is_classifier, is_regressor
models_names=['model1','model2','model3']
print("model name\t estimator name\t is_regressor\t is_classifier")    
for estimator , model_name in zip([model1,model2,model3],models_names):
        print("{}\t   {}     \t {}      \t {}".format(model_name,estimator.__class__.__name__,
                                              is_regressor(estimator),
                                              is_classifier(estimator)
                                              ))

model name	 estimator name	 is_regressor	 is_classifier
model1	   Sequential     	 False      	 False
model2	   Sequential     	 False      	 False
model3	   Sequential     	 False      	 False


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import numpy as np
models=[model1,model2,model3]
model_names=["model1","model2",'model3']
for model,model_name in zip(models,model_names):
            
    y_prob = model.predict(x_test)
    y_pred1 = y_prob.argmax(axis=-1)
    y_test1=np.argmax(y_test, axis=1)
    # accuracy
    print(model_name+" accuracy: ",accuracy_score(y_test1,y_pred1))
    del model , y_pred1, y_test1
    #Precision
    #print('Precision:', precision_score(y_test,y_pred1))
    #Recall
    #print('Recall:', recall_score(y_test1,y_pred1))
    #F-measure
    #print("Classification report")
    #print(classification_report(y_test1, y_pred1))

1/1 [==============================] - 18s 18s/step
model1 accuracy:  0.625
1/1 [==============================] - 6s 6s/step
model2 accuracy:  0.6875
1/1 [==============================] - 6s 6s/step
model3 accuracy:  0.09375


In [ ]:
# valid_set
x_test, y_test = valid_set.next() 

#VotingClassifier(hard)
estimators = [model1,model2,model3]
vc=votingClassifer(estimators=estimators,mode="hard",show_info="percent")
index_classes, class_names, probs=vc.predict(x_test)

1/1 [==============================] - 5s 5s/step


1/1 [==============================] - 5s 5s/step


1/1 [==============================] - 0s 92ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 93ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 93ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 98ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 92ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 92ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 93ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 94ms/step

 Each estimator predict a different class
1/1 [==============================] - 0s 93ms/step

 Each estimator predict a different class
percent: 100.00%

accuracy

In [ ]:
# inverse_to_categorical inverser format binary to format indexation
# datagenerator use the methode to_categorical for labelsation to frmat binary 
# the method inverce of to_categorical is argmax
import numpy as np
y_test1=np.argmax(y_test, axis=1)
y_pred1=index_classes

# accuracy
from sklearn.metrics import accuracy_score
print("votingClassifer(hard) accuracy : ",accuracy_score(y_test1,y_pred1))

votingClassifer(hard) accuracy :  0.53125
